In [1]:
from typing import List, Tuple, Union
import pandas as pd

In [2]:
class Purchase:
    """
    This class represents a purchase made by one or more people.
    """

    def __init__(self,
                 purchase_description: str,
                 amount_paid: float,
                 payer_name: tuple,
                 consumer: Union[str, Tuple[str]]
                ):
        """
        Initializes a Purchase object with the given purchase description, amount paid, payer name, and consumer(s).

        Parameters
        ----------
        purchase_description : str
            The description of the purchase.

        amount_paid : float
            The amount paid for the purchase.

        payer_name : str
            The name of the person who paid for the purchase.

        consumer : str or tuple of str
            The name(s) of the person(s) who consumed the purchase.
        """

        self.purchase_description = purchase_description
        self.amount_paid = amount_paid

        if isinstance(payer_name, str):
            payer_name = (payer_name, )
        self.payer_name = payer_name

        if isinstance(consumer, str):
            consumer = (consumer, )
        self.consumer = consumer



    def get_purchase_description(self) -> str:
        """
        Returns the description of the purchase.

        Returns
        -------
        str
            The description of the purchase.
        """
        return self.purchase_description



    def get_amount_paid(self) -> float:
        """
        Returns the amount paid for the purchase.

        Returns
        -------
        float
            The amount paid for the purchase.
        """
        return self.amount_paid



    def get_payer_name(self) -> str:
        """
         Returns the name of the person who paid amount for the purchase.

         Returns
         -------
         str
             The name of the person who paid amount for the purchase.
         """
        # Remove Name Duplicates
        payer_name = set(self.payer_name)
        return tuple(payer_name)



    def get_consumer(self) -> Tuple[str]:
        """
         Returns the name(s) of the person(s) who consumed the purchase.

         Returns
         -------
         tuple of str
             The name(s) of the person(s) who consumed the purchase.
         """

        # Remove Name Duplicates
        consumer = set(self.consumer)
        return tuple(consumer)

In [3]:
class PurchaseManager:
    """
    This class manages a list of purchases and can generate a data frame containing information about all purchases.
    """
     
    def __init__(self):
        # Initialize an empty list to store purchases
        self.purchases = []
          
    
    def add_purchase(self, purchase: Purchase):
        # Add a new purchase to the list
        self.purchases.append(purchase)
        return self.add_purchase

        
    def create_dataframe(self) -> pd.DataFrame:
        # Initialize empty lists to store data
        purchase_descriptions = []
        amounts_paid = []
        payer_names = []
        consumers = []
         
        # Loop through all purchases and extract data
        for purchase in self.purchases:
            purchase_descriptions.append(purchase.get_purchase_description())
            amounts_paid.append(purchase.get_amount_paid())
            payer_names.append(purchase.get_payer_name())
            consumers.append(purchase.get_consumer())
             
        # Create a dataset from extracted data
        dataset = {'purchase_description': purchase_descriptions,
                    'amount_paid': amounts_paid,
                    'payer_name': payer_names,
                    'consumer': consumers}

        # Create index labels for data frame
        index_labels = [f'Purchase {i+1}' for i in range(len(self.purchases))]
          
        # Create data frame from dataset and index labels
        data_frame = pd.DataFrame(dataset, index = index_labels)
             
        return data_frame


In [4]:
class DataProcessor:
    """
    This class processes a data frame containing information about purchases and can calculate:
    - total spent,
    - total share,
    - balances
    for each person involved in the purchases.
    """
    
    def __init__(self, data_frame: pd.DataFrame):
        # Initialize DataProcessor with given data frame
        self.data_frame = data_frame

        self.total_spent = self.calculate_total_spent()
        self.total_share = self.calculate_total_share()
        self.balances = self.calculate_balances()
        

    def calculate_total_spent(self) -> dict:
        # Initialize an empty dictionary to store total spent by each person
        total_spent = {}
           
        # Loop through all rows in data frame and calculate total spent by each person
        for index, row in self.data_frame.iterrows():
            split_amount = row['amount_paid'] / len(row['payer_name'])
            for payer in row['payer_name']:
                if payer not in total_spent:
                    total_spent[payer] = 0
                total_spent[payer] += split_amount
             
        # Round total spent values to 2 decimal places
        rounded_total_spent = {name: round(value, 2) for name, value in total_spent.items()}    
           
        return rounded_total_spent
     
     
    def calculate_total_share(self) -> dict:
        # Initialize an empty dictionary to store total share of each person
        total_share = {}
           
        # Loop through all rows in data frame and calculate total share of each person
        for index, row in self.data_frame.iterrows():
            split_amount = row['amount_paid'] / len(row['consumer'])
            for consumer in row['consumer']:
                if consumer not in total_share:
                       total_share[consumer] = 0
                total_share[consumer] += split_amount
         
        # Round total share values to 2 decimal places
        rounded_total_share = {name: round(value, 2) for name, value in total_share.items()}
           
        return rounded_total_share


    def calculate_balances(self) -> dict:
        # Initialize an empty dictionary to store balances of each person
        balances = {}
           
        # Initialize balances for all payer names
        for name in self.data_frame['payer_name'].unique():
            balances[name] = 0
               
        # Loop through all rows in data frame and calculate balances of each person
        for index, row in self.data_frame.iterrows():
            split_amount = row['amount_paid'] / len(row['consumer'])
            for consumer in row['consumer']:
                if consumer not in balances:
                    balances[consumer] = 0
                balances[consumer] -= split_amount
                
            for payer in row['payer_name']:
                balances[payer] += row['amount_paid'] / len(row['payer_name'])
             
        # Round balance values to 2 decimal places
        rounded_balances = {name: round(value, 2) for name, value in balances.items()}    
           
        return rounded_balances
    
    def create_summary_report_dataframe(self, ) -> pd.DataFrame:
        """
        Creates a summary report data frame containing information about
        - total spent
        - total share
        - balances
        for each person involved in the purchases.
          
        Parameters
        ----------
        data_frame : DataFrame
            A data frame containing information about the purchases.
        
        Returns
        -------
        DataFrame
            A data frame containing information about total spent, total share, and balances for each person.
        """
        
        # Extract unique names from the 'consumer' column in the data frame
        for column_name, column_data in self.data_frame.items():
            if column_name == 'consumer':
                column_consumer = column_data
            elif column_name == 'payer_name':
                colum_payer_name = column_data
                 
        # Create a set of unique names from the 'consumer and payer_name' column data        
        unique_names = set()
        # extract name from  each tuple 'column_consumer' and add name to 'unique_names' set
        for tuple_names in column_consumer:
            for name in tuple_names:
                unique_names.add(name)
        # extract name from  each tuple 'column_payer_name' and add name to 'unique_names' set       
        for tuple_name in colum_payer_name:
            for name in tuple_name:
                unique_names.add(name)
    
        # Initialize an empty dataset to store summary data
        summary_dataset = {name: [] for name in unique_names}

        # Add total spent data to dataset
        for name in unique_names:
            if name in self.total_spent:
                summary_dataset[name].append(self.total_spent[name])
            else:
                summary_dataset[name].append(0)

        # Add total share data to dataset
        for name in unique_names:
            if name in self.total_share:
                summary_dataset[name].append(self.total_share[name])
            else:
                summary_dataset[name].append(0)


        # Add balance data to dataset
        for name in unique_names:
            if name in self.balances:
                summary_dataset[name].append(self.balances[name])
            else:
                summary_dataset[name].append(0)


        # Create data frame from dataset with appropriate index labels
        summary_dataframe = pd.DataFrame(summary_dataset, index = ['TOTAL SPENT', 'TOTEL SHARE', 'BALANCES'])
         
        return summary_dataframe

{'Hasan'}


In [5]:
from typing import List, Tuple, Union

# Create a PurchaseManager object
purchase_manager = PurchaseManager()

# Create a list of unique names with proper capitalization(.title())
# names = Name().create_name_list('ali', 'reZA', 'sAbEr', 'ShagHayeGH')

# Create a Purchase object for fuel purchase
purchase_description = 'fuel'
amount_paid = 100
payer_name = 'Reza', 'Hejazizo'
consumer = 'Reza', 'Ali', 'Saber', 'Shaghayegh', 'Hejazizo'
# Create purchase1 object from class Purchase
purchase1 = Purchase(purchase_description, amount_paid, payer_name, consumer)
# Add (purchase1) object to the add_purcase from purchase_manager list of purchase
purchase_manager.add_purchase(purchase1)

# Create a Purchase object for car rent purchase
purchase_description = 'car rent'
amount_paid = 200
payer_name = 'Ali', 'Hejazizo'
consumer = 'Reza', 'Ali', 'Saber', 'Hejazizo'
purchase2 = Purchase(purchase_description, amount_paid, payer_name, consumer)
purchase_manager.add_purchase(purchase2)

# Create a Purchase object for house rent purchase
purchase_description = 'house rent'
amount_paid = 300
payer_name = 'Saber', 'Hejazizo'
consumer =  'Reza', 'Ali', 'Hejazizo'
purchase3 = Purchase(purchase_description, amount_paid, payer_name, consumer)
purchase_manager.add_purchase(purchase3)

# Create a Purchase object for food purchase
purchase_description = 'Food'
amount_paid = 400
payer_name = 'Shaghayegh', 'Hejazizo'
consumer = 'Reza', 'Hejazizo'
purchase4 = Purchase(purchase_description, amount_paid, payer_name, consumer)
purchase_manager.add_purchase(purchase4)

# Create a Purchase object for fruit purchase
purchase_description = 'Fruit'
amount_paid = 500
payer_name = 'Reza', 'Hejazizo'
consumer = 'Reza', 'Saber', 'Hejazizo'
purchase5 = Purchase(purchase_description, amount_paid, payer_name, consumer)
purchase_manager.add_purchase(purchase5)

# Create a Purchase object for supermarket purchase
purchase_description = 'Supermarket purchase'
amount_paid = 600
payer_name = 'Ali', 'Hejazizo'
consumer = 'Saber', 'Shaghayegh', 'Hejazizo'
purchase6 = Purchase(purchase_description, amount_paid, payer_name, consumer)
purchase_manager.add_purchase(purchase6)

# Create a data frame containing information about all purchases
data_frame = purchase_manager.create_dataframe()

# Create a DataProcessor object to process the dataframe
data_processor = DataProcessor(data_frame)
total_spent = data_processor.calculate_total_spent()
total_share = data_processor.calculate_total_share()
balances = data_processor.calculate_balances()
summary_report_dataframe = data_processor.create_summary_report_dataframe()

# Print the data frame and summary report
print(data_frame)
print()
print(f'total spent --> {total_spent}')
print(f'total share --> {total_share}')
print(f'Balances    --> {balances}')
print()
print(summary_report_dataframe)
print()



            purchase_description  amount_paid              payer_name  \
Purchase 1                  fuel          100        (Reza, Hejazizo)   
Purchase 2              car rent          200         (Ali, Hejazizo)   
Purchase 3            house rent          300       (Saber, Hejazizo)   
Purchase 4                  Food          400  (Shaghayegh, Hejazizo)   
Purchase 5                 Fruit          500        (Reza, Hejazizo)   
Purchase 6  Supermarket purchase          600         (Ali, Hejazizo)   

                                            consumer  
Purchase 1  (Reza, Ali, Shaghayegh, Saber, Hejazizo)  
Purchase 2              (Ali, Reza, Hejazizo, Saber)  
Purchase 3                     (Ali, Reza, Hejazizo)  
Purchase 4                          (Reza, Hejazizo)  
Purchase 5                   (Reza, Hejazizo, Saber)  
Purchase 6             (Shaghayegh, Saber, Hejazizo)  

total spent --> {'Reza': 300.0, 'Hejazizo': 1050.0, 'Ali': 400.0, 'Saber': 150.0, 'Shaghayegh': 200.0}

In [6]:
data_frame

,purchase_description,amount_paid,payer_name,consumer
Purchase 1,fuel,100,"(Reza, Hejazizo)","(Reza, Ali, Shaghayegh, Saber, Hejazizo)"
Purchase 2,car rent,200,"(Ali, Hejazizo)","(Ali, Reza, Hejazizo, Saber)"
Purchase 3,house rent,300,"(Saber, Hejazizo)","(Ali, Reza, Hejazizo)"
Purchase 4,Food,400,"(Shaghayegh, Hejazizo)","(Reza, Hejazizo)"
Purchase 5,Fruit,500,"(Reza, Hejazizo)","(Reza, Hejazizo, Saber)"
Purchase 6,Supermarket purchase,600,"(Ali, Hejazizo)","(Shaghayegh, Saber, Hejazizo)"


In [7]:
summary_report_dataframe

,Reza,Ali,Shaghayegh,Saber,Hejazizo
TOTAL SPENT,300.00,400.0,200.0,150.00,1050.00
TOTEL SHARE,536.67,170.0,220.0,436.67,736.67
BALANCES,-236.67,230.0,-20.0,-286.67,313.33
